In [95]:
#essential libraries

#from google_drive import gdrive

from matplotlib.pyplot import show
import numpy as np
import pandas as pd
from scipy.cluster.vq import *
import operator
#import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import scipy
from sklearn.feature_extraction import DictVectorizer
from matplotlib.backends.backend_pdf import PdfPages
import csv
import sys
import math
from copy import deepcopy, copy
import random
from datetime import datetime, timedelta
from operator import itemgetter
from itertools import chain
import os
import gc
import linecache
import multiprocessing
from functools import partial
from itertools import repeat
from IPython.display import clear_output
import json
from functools import reduce
import pdb

import pprint
pp = pprint.PrettyPrinter()

import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef

from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import OneClassSVM, SVC
from sklearn.mixture import GMM
from sklearn.mixture import DPGMM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy.cluster.hierarchy import fcluster
from scipy import spatial
from sklearn.multiclass import OneVsRestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as hier
from scipy import stats
from numpy.testing import assert_almost_equal

from scipy.spatial.distance import cosine as cosine_similarity

from divergence import gau_js as js_divergence
#from brick_parser import tagList, tagsetList, equipTagsetList, pointTagsetList, locationTagsetList,\
#pointTagList, equipTagList, locationTagList, equipPointDict
from brick_parser import equipTagsetList, pointTagsetList, locationTagsetList
tagsetList = list(set(equipTagsetList + pointTagsetList + locationTagsetList))
adder = lambda x,y: x + y
tagList = list(set(reduce(adder, [tagset.split('_') for tagset in tagsetList])))

#from cmu_parser import cmu_building_parse

debugFlag = False

In [96]:

#building_name = 'ap_m'
#building_name = 'bml'
#building_name = 'ghc'
#building_name = 'ebu3b'
building_name = 'ebu3b'
notUcsdBuildings = ['ghc']


In [97]:
import imp
import building_tokenizer
imp.reload(building_tokenizer)
import building_tokenizer as toker
from functools import reduce

tokenType = 'NoNumber'
if not building_name in notUcsdBuildings:
    naeDict = dict()
    naeDict['bonner'] = ["607", "608", "609", "557", "610"]
    naeDict['ap_m'] = ['514', '513','604']
    naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
    naeDict['ebu3b'] = ["505", "506"]
    naeDict['bml'] = ["510"]
    #naeDict['otterson'] = ["518", "517", "589", "590"]
    naeList = naeDict[building_name]

    labeledFile = 'metadata/' + building_name + '_sensor_types_location.csv'
    with open(labeledFile, 'rb') as fp:
        #truthDF = pd.read_excel(fp)
        truthDF = pd.DataFrame.from_csv(fp)
        #truthDF = truthDF.set_index(keys='Unique Identifier')

    wordFeatFile = 'data/wordfeat_'+building_name+'.pkl'

    tokenTypeList = ['NoNumber', 'Alphanumeric', 'AlphaAndNum', 'NumAsSingleWord']

    bacnetTypeMapDF = pd.DataFrame.from_csv('metadata/bacnettype_mapping.csv')
    unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')
    for val in Counter(unitMap.keys()).values():
        if val>1:
            "Unit map file ERROR"
            assert(False)
            
            
    trueDF = pd.DataFrame.from_csv('metadata/'+building_name+'_sensor_types_location.csv')
    sensorDF, nameList, jcinameList, descList, unitList, bacnettypeList, wordList = \
    toker.structure_metadata(buildingName=building_name, tokenType=tokenType, \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)

    origSensorDF = deepcopy(sensorDF)
    origNameList = deepcopy(nameList)
    origJcinameList = deepcopy(jcinameList)
    origDescList = deepcopy(descList)
    origUnitList = deepcopy(unitList)
    origBacnettypeList = deepcopy(bacnettypeList)
    origWordList = deepcopy(wordList)

    #_, rawNameList, rawJcinameList, rawDescList, _, _, _ = toker.structure_metadata(building_name=building_name, tokenType='AlphaAndNum', \
    _, rawNameList, rawJcinameList, rawDescList, _, _, _ =\
    toker.structure_metadata(buildingName=building_name, tokenType='JustSeparate', \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)
    
    
else: 
    filename = 'metadata/'+building_name+'_sensor_types_location.csv'
    #   filename = 'metadata/%s_sensor_types_location.csv'%building_name
    df = pd.read_csv(filename)
    sentenceDict = dict()    
    for i,raw in enumerate(df['bas_raw'].tolist()):
        sentenceDict[i] = toker.tokenize(tokenType, raw)
    
    
## Common part
    
#with open('data/'+building_name+'_str_score_dict.pkl', 'rb') as fp:
#    scoreDictDict = pickle.load(fp)
#    ### If a word is exactly matched with another, fix it.
#    for word, scoreDict in scoreDictDict.items():
#        if word in scoreDict.keys():
#            scoreDictDict[word] = {word:1}
#    scoreDictDict['co'] = {'co2':1}
        
adder = lambda x,y:x+y
splitter = lambda x:x.split()
bacnetTypes = list(set(reduce(adder,map(splitter,origBacnettypeList),[])))
#for bacnetType in bacnetTypes:
#    scoreDictDict[bacnetType] = {bacnetType:1}
#units = list(set(unitMap['word'].tolist()))
#for unit in units:
#    if type(unit)==str:
#        scoreDictDict[unit] = {unit:1}

In [98]:
with open("metadata/%s_sentence_dict_justseparate.json"%building_name, "r") as fp:
    sentenceDict = json.load(fp)

In [99]:
def check_re_rules(word, phrase, re_dict):
    re_list = re_dict.get(word)
    if re_list:
        for (re_rule, label) in re_list:
            if re.findall(re_rule, phrase):
                return label
    return None

def validate_label(label):
    nonBrickTagList = ['none', 'rightidentifier', 'leftidentifier', 'pump_flow_status',
                       'networkadapter', 'vnd', 'name', 'unknown']
    # name: stand alone unique name.
    # left identifier: contraints meaning of left tagset
    # right identifier: contraints meaning of right tagset
    label = label.split('-')[0]
    if label in nonBrickTagList:
        return True
    token_list = label.split('_')
    valid_label_list = tagList + tagsetList
    for token in token_list:
        valid_label_list = [label for label in valid_label_list if token in label]
    if len(valid_label_list)>0:
        return True
    else:
        return False
        

def ask_expert_labels(sentence):
    # Input: List of words
    # Output: feature list:
    # Human operation: Give labels on the words.
    sliceSize = 2
    featureDict = dict()

    #for words in rolling_window(sentence, 2):
    #    for word in wo
    print("Given sentence: %s"%sentence)
    labelList = list()
    for i, word in enumerate(sentence):
        if re.match('[^0-9a-zA-Z]+', word):
            labelList.append('none')
            continue
        if re.match('\d+', word):
            labelList.append('leftidentifier')
            continue
        if word in known_word_list:
            labelList.append(known_word_dict[word])
            continue
        start_idx = max(i-2, 0)
        end_idx = min(i+3, len(sentence))
        phrase = ''.join(sentence[start_idx:end_idx])
        found_by_re = check_re_rules(word, phrase, re_dict=re_dict)
        if found_by_re:
            labelList.append(found_by_re)
            continue
        if word in tagList:
            labelList.append(word)
            continue
        while True:
            label = input(str(word)+': \n')
            if validate_label(label):
                labelList.append(label)
                break
            else:
                print("Pick Tag among the following tags: " + str(tagList))
    
    # compensate left->right_identifier of numbers with rd, st, th
    for i, (word, label) in enumerate(zip(sentence, labelList)):
        if i < len(sentence)-1:
            if word.isdigit() and sentence[i+1] in ['rd', 'nd', 'st', 'th']:
                labelList[i] = 'rightidentifier'           
    
    return labelList

In [108]:
labelFilename = 'metadata/%s_label_dict_justseparate.json'%building_name
if os.path.isfile(labelFilename):
    with open(labelFilename, 'r') as fp:
        labelListDict = json.load(fp)
else:
    labelListDict = {}
with open(labelFilename+'.backup', 'w') as fp:
    json.dump(labelListDict, fp, indent=2)

In [101]:
with open('metadata/word_dict.json', 'r') as fp:
    known_word_dict = json.load(fp)[building_name]
known_word_list = known_word_dict.keys()
with open('metadata/re_dict.json', 'r') as fp:
    re_dict = json.load(fp)[building_name]

In [117]:
labelSampleNum = 1000

srcids = sentenceDict.keys()
if building_name == 'ghc':
    for srcid in random.sample(srcids, labelSampleNum):
        if not srcid in labelListDict.keys():
            print(srcid)
            labelListDict[srcid] = ask_expert_labels(sentenceDict[srcid])
            print('\n')
else:    
    #for i in random.sample(range(0,len(sensorDF)), labelSampleNum):
        #srcid = sensorDF.iloc[i].name
    for srcid in random.sample(srcids, labelSampleNum):
        if not srcid in labelListDict.keys():
            print(srcid)
            labelListDict[srcid] = ask_expert_labels(sentenceDict[srcid])


506_1_3002925
Given sentence: ['nae', ' ', '06', ' ', 'n', '2', ' ', '2', ' ', 'vma', '104', ' ', 'dpr', ' ', 'c', '\n', 'ebu3b', '.', 'rm', '-', '3138', '.', '.', 'dpr', '-', 'c', '\n', 'damper', ' ', 'command', '\n']
505_1_3001619
Given sentence: ['nae', ' ', '05', ' ', 'n', '2', ' ', '2', ' ', 'vma', ' ', '151', ' ', 'htg', 'pid', ' ', 'o', '\n', 'ebu3b', '.', 'rm', '-', '1115', '.', '.', 'htg', 'pid', '-', 'o', '\n', 'heating', ' ', 'command', '\n']
506_14_3002326
Given sentence: ['nae', ' ', '06', ' ', 'n', '2', ' ', '2', ' ', 'vma', '155', ' ', 'occ', ' ', 'cmd', '\n', 'ebu3b', '.', 'rm', '-', '4203', '.', '.', 'occ', '-', 'cmd', '\n', 'occupied', ' ', 'command', '\n']
506_1_3000978
Given sentence: ['nae', ' ', '06', ' ', 'n', '2', ' ', '1', ' ', 'vma', '173', ' ', 'clg', 'min', 'flo', '\n', 'ebu3b', '.', 'rm', '-', '3246', '.', '.', 'clg', 'min', 'flo', '\n', 'occupied', ' ', 'clg', ' ', 'min', '\n']
505_1_3026019
Given sentence: ['nae', ' ', '05', ' ', 'n', '2', ' ', '2', ' ', 

c: 
command
505_3_3015518
Given sentence: ['nae', ' ', '05', ' ', 'n', '2', ' ', '1', ' ', 'vma', ' ', '117', ' ', 'occ', ' ', 'sen', '\n', 'ebu3b', '.', 'rm', '-', 'b', '220', '.', '.', 'occ', '-', 'sen', '\n', 'occupancy', ' ', 'sensor', '\n']
506_1_3001396
Given sentence: ['nae', ' ', '06', ' ', 'n', '2', ' ', '1', ' ', 'vma', '112', ' ', 'common', 'sp', '\n', 'ebu3b', '.', 'rm', '-', '2226', '.', '.', 'common', 'sp', '\n', 'common', ' ', 'setpoint', '\n']
505_0_3000473
Given sentence: ['nae', ' ', '05', ' ', 'n', '2', ' ', '1', ' ', 'nae', '5', ' ', 't', '1', ' ', 'vnd', '41', ' ', ' ', 'sf', ' ', '1', ' ', 'vfd', ' ', '41', ' ', 'current', '\n', 'ebu3b', '.', 'sf', '1', '.', 'current', '\n', 'current', '\n']
506_1_3015777
Given sentence: ['nae', ' ', '06', ' ', 'n', '2', ' ', '2', ' ', 'vma', '160', ' ', 'w', ' ', 'c', ' ', 'adj', '\n', 'ebu3b', '.', 'rm', '-', '4136', '.', '.', 'w', '-', 'c', '-', 'adj', '\n', 'warm', '/', 'cool', ' ', 'adjust', '\n']
506_2_3003387
Given sentence

t: 
temperature
t: 
temperature
505_3_3015308
Given sentence: ['nae', ' ', '05', ' ', 'n', '2', ' ', '2', ' ', 'vnd', ' ', '162', ' ', 'off', ' ', 'locl', '\n', 'ebu3b', '.', 'crac', '-', '1', '.', 'off', '-', 'locl', '\n', 'unit', ' ', 'turned', ' ', 'off', ' ', 'locally', '\n']
505_3_3015124
Given sentence: ['nae', ' ', '05', ' ', 'n', '2', ' ', '1', ' ', 'dx', ' ', '12', ' ', 'file', ' ', 'server', ' ', 'chw', ' ', 'system', ' ', 'chp', '4', ' ', 'sts', '\n', 'ebu3b', '.', 'fs', 'chw', '.', 'chp', '4', '-', 'sts', '\n', 'chilled', ' ', 'water', ' ', 'pump', ' ', '4', ' ', 'status', '\n']
506_1_3002239
Given sentence: ['nae', ' ', '06', ' ', 'n', '2', ' ', '2', ' ', 'vma', '151', ' ', 'htg', 'pid', ' ', 'o', '\n', 'ebu3b', '.', 'rm', '-', '4254', '.', '.', 'htg', 'pid', '-', 'o', '\n', 'heating', ' ', 'command', '\n']
506_1_3000755
Given sentence: ['nae', ' ', '06', ' ', 'n', '2', ' ', '1', ' ', 'vma', '162', ' ', 'clg', 'min', 'flo', '\n', 'ebu3b', '.', 'rm', '-', '3207', '.', '.', 

In [118]:
print(len(labelListDict))
with open(labelFilename, 'w') as fp:
    json.dump(labelListDict,fp, indent=4, separators=(',', ': '))

1084


In [104]:
wrongSrcidList = list()
for srcid, sentence in sentenceDict.items():
    if 't' in sentence and '5' in sentence:
        #wrongSrcidList.append(srcid)
        if srcid in labelListDict.keys():
            print(srcid, sentence)

505_0_3000377 ['nae', ' ', '05', ' ', 'n', '2', ' ', '1', ' ', 'nae', '5', ' ', 't', '1', ' ', 'vnd', '23', ' ', ' ', 'hwp', '1', ' ', 'vfd', ' ', '23', ' ', 'freq', ' ', 'output', '\n', 'ebu3b', '.', 'hwp', '1', '.', 'freq', '-', 'output', '\n', 'frequency', ' ', 'output', '\n']
505_0_3000553 ['nae', ' ', '05', ' ', 'n', '2', ' ', '1', ' ', 'nae', '5', ' ', 't', '1', ' ', 'vnd', '43', ' ', ' ', 'sf', ' ', '3', ' ', 'vfd', ' ', '43', ' ', 'current', '\n', 'ebu3b', '.', 'sf', '3', '.', 'current', '\n', 'current', '\n']
505_0_3000606 ['nae', ' ', '05', ' ', 'n', '2', ' ', '1', ' ', 'nae', '5', ' ', 't', '1', ' ', 'vnd', '44', ' ', ' ', 'sf', ' ', '4', ' ', 'vfd', ' ', '44', ' ', 'ai', '2', ' ', 'actual', '\n', 'ebu3b', '.', 'sf', '4', '.', 'ai', '2', '-', 'actual', '\n', 'ai', ' ', '2', ' ', 'actual', '\n']
505_1_3000332 ['nae', ' ', '05', ' ', 'n', '2', ' ', '1', ' ', 'nae', '5', ' ', 't', '1', ' ', 'vnd', '075', ' ', ' ', 'chw', 'p', '1', ' ', 'vfd', ' ', '21', ' ', 'current', ' ', 'li